In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid')


data_train_path = (r"/content/drive/My Drive/Colab Notebooks/Verzeo Internship/Major Project/datasets/Data_Train.xlsx")
data_test_path = (r"/content/drive/My Drive/Colab Notebooks/Verzeo Internship/Major Project/datasets/Data_Test.xlsx")

data_train = pd.read_excel(data_train_path)
data_test = pd.read_excel(data_test_path)
data_train.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


## The Data-Preprocessing part

In [2]:
df1 = data_train[data_train.isna().any(axis=1)]
df2 = data_test[data_test.isna().any(axis=1)]

# Null present as string in the data 
data_test_null_string = data_test[data_test['Power'].str.contains('null' , na = True)]


columns_to_be_filled_with_NaN = ["Power"]

def fill_columns_with_NaN(columns_to_be_filled_with_NaN):
    for i in range (len(columns_to_be_filled_with_NaN)):    
        data_train[columns_to_be_filled_with_NaN[i]] = data_train[columns_to_be_filled_with_NaN[i]].replace(to_replace="[null]" , value = np.NaN , regex = True)
        data_test[columns_to_be_filled_with_NaN[i]] = data_test[columns_to_be_filled_with_NaN[i]].replace(to_replace="[null]" , value = np.NaN , regex = True)

fill_columns_with_NaN(columns_to_be_filled_with_NaN)

empty_cols_list_train = ["Mileage","Engine","Power","Seats"] 
empty_cols_list_test = ["Engine","Power","Seats"] 


def my_Nan_filling_function(dataset , empty_cols_list):
    for i in range(len(empty_cols_list)):
        
        # Replacing Nan with mean values
        # dataset[empty_cols_list[i]].fillna(dataset[empty_cols_list[i]].mean , inplace = True)
        
        # Replacing Nan with mode values
        dataset[empty_cols_list[i]].fillna(dataset[empty_cols_list[i]].mode()[0] , inplace = True)

    print(dataset)

my_Nan_filling_function(data_train , empty_cols_list_train)

my_Nan_filling_function(data_test , empty_cols_list_test)

list_of_columns_with_units = ["Mileage" , "Engine" , "Power"] 

def remove_units_from_columns(list_of_columns_with_units):
    for i in range(len(list_of_columns_with_units)):
        data_train[list_of_columns_with_units[i]] = data_train[list_of_columns_with_units[i]].str.split(" " , expand = True)
        data_train[list_of_columns_with_units[i]] = data_train[list_of_columns_with_units[i]].astype("float")
        data_test[list_of_columns_with_units[i]] = data_test[list_of_columns_with_units[i]].str.split(" " , expand = True)
        data_test[list_of_columns_with_units[i]] = data_test[list_of_columns_with_units[i]].astype("float")

remove_units_from_columns(list_of_columns_with_units)

data_train["Year"] = data_train["Year"].astype("int32" , copy = False)
data_test["Year"] = data_test["Year"].astype("int32" , copy = False)

                                  Name    Location  ...  Seats  Price
0               Maruti Wagon R LXI CNG      Mumbai  ...    5.0   1.75
1     Hyundai Creta 1.6 CRDi SX Option        Pune  ...    5.0  12.50
2                         Honda Jazz V     Chennai  ...    5.0   4.50
3                    Maruti Ertiga VDI     Chennai  ...    7.0   6.00
4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  ...    5.0  17.74
...                                ...         ...  ...    ...    ...
6014                  Maruti Swift VDI       Delhi  ...    5.0   4.75
6015          Hyundai Xcent 1.1 CRDi S      Jaipur  ...    5.0   4.00
6016             Mahindra Xylo D4 BSIV      Jaipur  ...    8.0   2.90
6017                Maruti Wagon R VXI     Kolkata  ...    5.0   2.65
6018             Chevrolet Beat Diesel   Hyderabad  ...    5.0   2.50

[6019 rows x 12 columns]
                                                   Name  ... Seats
0                               Maruti Alto K10 LXI CNG  ...   4.0


## Removing the outliers part

In [3]:
def get_numerical_features(dataset):
    df = dataset
    df_numerics_only = df.select_dtypes(include=np.number)
    # print(df_numerics_only)
    colnames_numerics_only = df.select_dtypes(include=np.number).columns.tolist()
    # print(colnames_numerics_only)
    return colnames_numerics_only

df_train = get_numerical_features(data_train)
print(df_train)

df_test = get_numerical_features(data_test)
print(df_test)

print(data_train.shape)

remove_train_outliers_list = df_train
remove_test_outliers_list = df_test

def remove_outliers(dataset , remove_outliers_list):
    # Iterating across the list of numerical categories for which the outliers have to be removed
    for i in range(len(remove_outliers_list)):
        # Change the value of the outlier criterion for better results
        dataset.drop(dataset[dataset[remove_outliers_list[i]] >=( 0.99999 * dataset[remove_outliers_list[i]].max()) ].index, inplace = True )

remove_outliers(data_train , remove_train_outliers_list)
remove_outliers(data_test , remove_test_outliers_list)

print(data_train.shape)

['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price']
['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats']
(6019, 12)
(5903, 12)


## Skipping the EDA part (in the other notebook)

In [4]:
"""
 Summary of some of the vairables
"""

'''Actual train data '''
# data_train  
'''Actual train data xlsx file '''
# data_train_path   
'''Actual train data csv file '''
# data_train_csv_path 
''' Only Numerical Features of train data  '''
# df_train  


'''Actual test data '''
# data_test  
'''Actual test data xlsx file '''
# data_test_path   
'''Actual test data csv file '''
# data_test_csv_path 
''' Only Numerical Features of test data  '''
# df_test  


' Only Numerical Features of test data  '

# Building the model
 

In [5]:
data_train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74


In [6]:
data_test.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.70,796.0,47.30,5.0
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68,2393.0,147.80,7.0
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59,1364.0,74.00,5.0
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.50,1197.0,82.85,5.0
5,Mahindra XUV500 W8 2WD,Coimbatore,2016,85609,Diesel,Manual,Second,16.00,2179.0,140.00,7.0


In [7]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5903 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               5903 non-null   object 
 1   Location           5903 non-null   object 
 2   Year               5903 non-null   int32  
 3   Kilometers_Driven  5903 non-null   int64  
 4   Fuel_Type          5903 non-null   object 
 5   Transmission       5903 non-null   object 
 6   Owner_Type         5903 non-null   object 
 7   Mileage            5903 non-null   float64
 8   Engine             5903 non-null   float64
 9   Power              5903 non-null   float64
 10  Seats              5903 non-null   float64
 11  Price              5903 non-null   float64
dtypes: float64(5), int32(1), int64(1), object(5)
memory usage: 576.5+ KB


In [8]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1208 entries, 1 to 1233
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1208 non-null   object 
 1   Location           1208 non-null   object 
 2   Year               1208 non-null   int32  
 3   Kilometers_Driven  1208 non-null   int64  
 4   Fuel_Type          1208 non-null   object 
 5   Transmission       1208 non-null   object 
 6   Owner_Type         1208 non-null   object 
 7   Mileage            1208 non-null   float64
 8   Engine             1208 non-null   float64
 9   Power              1208 non-null   float64
 10  Seats              1208 non-null   float64
dtypes: float64(4), int32(1), int64(1), object(5)
memory usage: 108.5+ KB


In [9]:
categorical_columns_only =['Location', 'Fuel_Type','Transmission', 'Owner_Type', 'Seats']
len(categorical_columns_only)

5

In [10]:
# Creating a Final Dataframe which conatains both training and testing datasets
# so that we don't miss any categories which might be present in one dataset but not in the other.

data_test.to_csv('formulatedtest.csv',index=False)

test_data = pd.read_csv('formulatedtest.csv')

final_data=pd.concat([data_train,test_data],axis=0)

final_data.shape

(7111, 12)

In [11]:
final_data['Price']

0        1.75
1       12.50
2        4.50
3        6.00
4       17.74
        ...  
1203      NaN
1204      NaN
1205      NaN
1206      NaN
1207      NaN
Name: Price, Length: 7111, dtype: float64

In [12]:
def categorize_into_onehot_multicolumns(mulitcolumns):
    data_final=final_data
    i=0
    for fields in mulitcolumns:
        print(fields)
        data_dummy=pd.get_dummies(final_data[fields],drop_first=True)
        final_data.drop([fields],axis=1,inplace=True)
        if i==0:
            data_final=data_dummy.copy()
        else:
            data_final=pd.concat([data_final,data_dummy],axis=1)
        i=i+1
    data_final=pd.concat([final_data,data_final],axis=1)
        
    return data_final

final_data = categorize_into_onehot_multicolumns(categorical_columns_only)

Location
Fuel_Type
Transmission
Owner_Type
Seats


In [13]:
final_data.shape

(7111, 32)

In [14]:
final_data.head()

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Price,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Diesel,Electric,LPG,Petrol,Manual,Fourth & Above,Second,Third,2.0,4.0,5.0,6.0,7.0,8.0,9.0
0,Maruti Wagon R LXI CNG,2010,72000,26.60,998.0,58.16,1.75,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,1582.0,126.20,12.50,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,Honda Jazz V,2011,46000,18.20,1199.0,88.70,4.50,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
3,Maruti Ertiga VDI,2012,87000,20.77,1248.0,88.76,6.00,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,1968.0,140.80,17.74,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [15]:
# Removing Duplicate Features (just in case there are any)
final_data =final_data.loc[:,~final_data.columns.duplicated()]
final_data.shape

(7111, 32)

In [16]:
final_data

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Price,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Diesel,Electric,LPG,Petrol,Manual,Fourth & Above,Second,Third,2.0,4.0,5.0,6.0,7.0,8.0,9.0
0,Maruti Wagon R LXI CNG,2010,72000,26.60,998.0,58.16,1.75,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,2015,41000,19.67,1582.0,126.20,12.50,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,Honda Jazz V,2011,46000,18.20,1199.0,88.70,4.50,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
3,Maruti Ertiga VDI,2012,87000,20.77,1248.0,88.76,6.00,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,Audi A4 New 2.0 TDI Multitronic,2013,40670,15.20,1968.0,140.80,17.74,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,Volkswagen Vento Diesel Trendline,2011,89411,20.54,1598.0,103.60,NaN,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1204,Volkswagen Polo GT TSI,2015,59000,17.21,1197.0,103.60,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1205,Nissan Micra Diesel XV,2012,28000,23.08,1461.0,63.10,NaN,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1206,Volkswagen Polo GT TSI,2013,52262,17.20,1197.0,103.60,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0


In [0]:
final_data_train=final_data.iloc[:5903,:]
final_data_test=final_data.iloc[5903:,:]

In [18]:
final_data_train.sample(10)

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Price,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Diesel,Electric,LPG,Petrol,Manual,Fourth & Above,Second,Third,2.0,4.0,5.0,6.0,7.0,8.0,9.0
4608,BMW X5 3.0d,2009,150000,11.00,2993.0,235.00,18.75,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4090,Hyundai Verna 1.6 CRDi SX,2015,62542,23.90,1582.0,126.20,6.50,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0
3259,Audi Q3 30 TDI Premium FWD,2017,31000,18.51,1968.0,147.51,27.00,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3679,Mercedes-Benz New C-Class C 220 CDI Style,2017,10000,19.27,2143.0,167.62,36.00,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4524,Hyundai Santa Fe 4X4,2011,148000,14.66,2199.0,194.30,8.90,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0
5307,Maruti Baleno Zeta 1.2,2018,13000,21.40,1197.0,83.10,6.60,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
3649,Tata Indigo LS,2008,300000,17.00,1405.0,70.00,1.00,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2104,BMW 3 Series 2005-2011 325i Sedan,2010,77849,10.90,2497.0,215.00,11.27,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
5091,Chevrolet Aveo 1.6 LT,2007,63000,14.20,1598.0,102.00,1.20,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0
4823,Volkswagen Polo Petrol Comfortline 1.2L,2012,64806,16.47,1198.0,73.90,3.65,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0


In [19]:
final_data_test.sample(10)

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Price,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Diesel,Electric,LPG,Petrol,Manual,Fourth & Above,Second,Third,2.0,4.0,5.0,6.0,7.0,8.0,9.0
843,BMW 5 Series 520d Sedan,2009,95000,18.48,1197.0,74.00,NaN,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
592,Mercedes-Benz CLA 200 D Sport Edition,2017,41372,17.90,2143.0,136.00,NaN,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
890,Maruti Swift Dzire 1.2 Lxi BSIV,2010,122000,17.50,1197.0,85.80,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
389,Hyundai Verna CRDi ABS,2007,89564,18.10,1493.0,110.00,NaN,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0
1038,Maruti Ciaz RS ZDi Plus SHVS,2017,45158,28.09,1248.0,88.50,NaN,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0
135,Ford EcoSport 1.0 Ecoboost Titanium Plus,2014,56000,18.88,999.0,123.24,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
70,Tata Tiago AMT 1.2 Revotron XTA,2017,14000,23.84,1199.0,84.00,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
758,Toyota Innova 2.5 GX (Diesel) 8 Seater,2014,173000,12.99,2494.0,100.60,NaN,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0
539,Skoda Rapid 1.6 MPI AT Style,2016,18856,14.30,1598.0,103.52,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
568,Mahindra Xylo H9,2014,101000,14.02,2179.0,118.30,NaN,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [20]:
# Removing the Used Car Price Column 
final_data_test.drop(['Name','Price'],axis=1,inplace=True)
X_train=final_data_train.drop(['Name','Price'],axis=1)
# Making it the Target variable
y_train=final_data_train['Price']

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Using XGBoost Regressor for Used-Car Price Prediction

In [21]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost
regressor=xgboost.XGBRegressor()

booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

## Hyper Parameter Optimization
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   32.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  4.9min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  8.4min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 12.3min finished


[17:32:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=...
                   iid='deprecated', n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['g

In [22]:
random_cv.best_estimator_

XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=2, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
# DEPRECATED
# xgboost_regressor = xgboost.XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, gamma=0,
#              importance_type='gain', learning_rate=0.1, max_delta_step=0,
#              max_depth=5, min_child_weight=2, missing=None, n_estimators=1100,
#              n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
#              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#              silent=None, subsample=1, verbosity=1)
xgboost_regressor = xgboost.XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=2, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [28]:
xgboost_regressor.fit(X_train,y_train)

XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=2, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

In [32]:
final_data_test.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Bangalore,Chennai,Coimbatore,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mumbai,Pune,Diesel,Electric,LPG,Petrol,Manual,Fourth & Above,Second,Third,2.0,4.0,5.0,6.0,7.0,8.0,9.0
0,2013,54493,24.70,796.0,47.30,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0
1,2017,34000,13.68,2393.0,147.80,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,2012,139000,23.59,1364.0,74.00,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,2014,29000,18.50,1197.0,82.85,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
4,2016,85609,16.00,2179.0,140.00,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0


In [34]:
y_pred=regressor.predict(final_data_test)
y_pred

array([ 2.9966729, 16.336628 ,  4.3285174, ...,  2.6529317,  5.725777 ,
       19.476828 ], dtype=float32)

In [36]:
y_pred.shape

(1208,)

In [42]:
data_submission = data_test

y_pred_df=pd.DataFrame(y_pred)

datasets = pd.concat([data_submission['Name'],y_pred_df] , axis = 1)
datasets.columns = ['Name' , 'Price']
datasets.to_csv('sample_submission1.csv',index=False)
check_datasets = pd.read_csv('sample_submission1.csv')
check_datasets.sample(10)

,Name,Price
200,Maruti Dzire ZDI,39.457115
1017,Hyundai Santro Xing GLS,6.919238
781,Hyundai i20 1.4 CRDi Magna,5.924447
1035,Hyundai i20 1.2 Sportz,9.546096
81,Toyota Camry Hybrid 2.5,8.596716
244,Hyundai Xcent 1.2 Kappa S Option,7.481634
1018,Ford Ecosport 1.5 DV5 MT Ambiente,12.599196
330,Volkswagen Jetta 2007-2011 1.6 Trendline,1.743509
853,Tata Indica Vista Aura 1.3 Quadrajet,9.696537
158,Mercedes-Benz M-Class ML 350 4Matic,8.138403
